## Demo notebook for the mcp exchange rate server - used to show usaage of mcp_exchange_rate.py


### see the README.md file for instructions on how to get the free api key
### uses claude sonnet 4 openapi endpoint ="https://api.anthropic.com/v1"

In [ ]:
import os
import asyncio
from agents import Agent, OpenAIChatCompletionsModel, Runner, trace
from agents.mcp import MCPServerStdio
from openai import AsyncOpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display



load_dotenv(override=True)

In [ ]:


# 1. Configure Claude client 
# make sure t=you have the antropic key in your .env file
claude_client = AsyncOpenAI(
    base_url="https://api.anthropic.com/v1",
    api_key=os.getenv("ANTHROPIC_API_KEY")
)

# 2. Create Claude model
claude_model = OpenAIChatCompletionsModel(
    model="claude-sonnet-4-20250514",
    openai_client=claude_client
)
# MCP server for forex exchange rates
exchange_rate_params = {
    "command": "uv", "args": ["run", "mcp_exchange_rates.py"]
}
mcp_tools="None"

print(exchange_rate_params)
# MCP server for forex exchange rates call



## Run the exchnage rate server to get various cross rates

In [ ]:
try:
    async with MCPServerStdio(
        params=exchange_rate_params, client_session_timeout_seconds=30) as forex_server:
        print("✅ Connected to Forex MCP server")
       # 4. Connect to MCP server
                
        # 5. List available tools
        forex_tools = await forex_server.list_tools()
        tools = forex_tools
        print(f"�� Available tools: {tools}")
        
        instructions="""You are a financial research assistant. You have access to usd conversion tools.  
        always display information on fx rates in a table format and the date of the exhcnge rate quote"""
        
        request ="provide the latest exchange rate of usd/aud usd/eur, usd/nzd, usd/gbp, usd/can - print the rates in a table format\
                    print the inverse rates of the above rates"
                
        # 6. Create Claude agent with MCP server
        agent = Agent(
            name="claude_researcher",
            instructions=instructions,
            model=claude_model,
            mcp_servers=[forex_server]
        )
        
        print("✅ Claude agent created with MCP servers")
        
        
        # 7. Use the agent with tracing
        with trace("Claude Web Research"):
            result = await Runner.run(
                agent,
                request,
                max_turns=25
                )
        
        print("\n📊 Claude Agent Response:")
        print("=" * 50)
        # print(result.final_output)
        display(Markdown(result.final_output))
        print("=" * 50)
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
